In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
from tqdm import tqdm
import csv

## Save numpy file

In [47]:
def save_npy(data, fname):
    with open(os.getcwd() + f"/../data/{fname}.npy", 'wb') as f:
        np.save(f, data)

## Data Loading

In [3]:
DD = "/home/hwixley/Data/stoic2021/data/mha/"

In [4]:
files = os.listdir(DD)
num_files = len(files)
print(num_files)

2000


# Parse Labels

In [11]:
labels = np.zeros((num_files, 3))

In [25]:
with open(DD + '../../metadata/reference.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    
    for i, row in enumerate(spamreader):
        if i > 0:
            labels[i-1,:] = row[0].split(",")

In [26]:
labels

array([[8.6220e+03, 1.0000e+00, 0.0000e+00],
       [1.0720e+03, 1.0000e+00, 0.0000e+00],
       [1.0456e+04, 1.0000e+00, 0.0000e+00],
       ...,
       [8.3900e+02, 0.0000e+00, 0.0000e+00],
       [1.0300e+04, 0.0000e+00, 0.0000e+00],
       [9.6760e+03, 1.0000e+00, 0.0000e+00]])

In [29]:
with open(os.getcwd() + "/../data/labels.npy", 'wb') as f:
    np.save(f, labels)

In [45]:
labels[0,:]

array([8.622e+03, 1.000e+00, 0.000e+00])

In [ ]:
labels

# Slicing CT scans

In [55]:
import math

In [ ]:
#sl_thickness = 2

for i in tqdm(range(labels.shape[0])):
    label = labels[i,:]
    print(label[0])
    inputImage = sitk.ReadImage(DD + str(int(label[0])) + ".mha")
    
    ct_image = np.zeros((inputImage.GetDepth(),512,512))
    for d in tqdm(range(inputImage.GetDepth())):
        for row in range(512):
            ct_image[d,row,:] = [inputImage.GetPixel([row,col,d]) for col in range(512)]
              
    save_npy(ct_slice, f"npy-images/{str(int(label[0]))}")

  0%|                                                  | 0/2000 [00:00<?, ?it/s]

8622.0



 26%|██████████▋                               | 96/376 [00:26<01:18,  3.55it/s]


 51%|█████████████████████                    | 193/376 [00:54<00:52,  3.50it/s]


 77%|███████████████████████████████▌         | 290/376 [01:21<00:24,  3.56it/s]


...

7310.0



 23%|█████████▌                                | 96/422 [00:27<01:33,  3.49it/s]


 46%|██████████████████▊                      | 193/422 [00:54<01:04,  3.57it/s]


 69%|████████████████████████████▏            | 290/422 [01:21<00:37,  3.57it/s]


 75%|██████████████████████████████▊          | 317/422 [01:29<00:29,  3.57it/s]

## Sliding Window Slices

In [ ]:
window_size = 2

for i in tqdm(range(labels.shape[0])):
    label = labels[i,:]
    inputImage = sitk.ReadImage(DD + str(int(label[0])) + ".mha")
    
    for d in tqdm(range(inputImage.GetDepth())):
        if (d + window_size - 1) < inputImage.GetDepth():
            ct_slice = np.zeros((window_size,512,512))
            
            for w in range(window_size):
                for row in range(512):
                    ct_slice[w,row,:] = [inputImage.GetPixel([row,col,d+w]) for col in range(512)]

            save_npy(ct_slice, f"window-slices/{str(int(label[0]))}-{d}")
        else:
            break